In [ ]:
from diffusers import DiffusionPipeline
from diffusers.callbacks import PipelineCallback
import matplotlib.pyplot as plt
import torch

In [ ]:
def display_slider_images(images, titles):
    fig, axes = plt.subplots(1, len(images), figsize=(len(images)*3, 3))
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if len(images) == 1:
            ax = axes
        else:
            ax = axes[i]
        ax.imshow(img)
        ax.set_title(title)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
class ConceptSliderCallback(PipelineCallback):
    """
    Enable Concept Slider after certain number of steps (set by `slider_strength`), this callback will set the LoRA scale to `0.0` or `slider_scale` based on the strength.

    Use strength < 1 if you want more precise edits (recommend: .7 - .9)
    """
    tensor_inputs = []

    def __init__(self, slider_strength=1, slider_names=None, slider_scales=[0]):
        super().__init__()
        self.slider_names = slider_names
        self.slider_scales = slider_scales
        self.slider_strength = slider_strength
    
    def callback_fn(self, pipeline, step_index, timestep, callback_kwargs):
        # Use cutoff_step_index if it's not None, otherwise use cutoff_step_ratio
        attach_step = (
           pipeline.num_timesteps - int(pipeline.num_timesteps * self.slider_strength)
        )


        # at the attach_step point start adding the slider
        if step_index == attach_step:
            pipe.set_adapters(self.slider_names, adapter_weights=self.slider_scales)

        # after final step set the slider to 0 (there is a better implementation if we  callback_at_beginning of step exists in diffusers) 
        if step_index == pipeline.num_timesteps-1:
            pipe.set_adapters(self.slider_names, adapter_weights=[0.]*len(self.slider_names))
        
        return callback_kwargs


In [ ]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.bfloat16)
pipe.to("cuda").to(torch.bfloat16)

# you can use your trained slider (either .pt or .safetensors file with diffusers)
adapter_path = 'models/sdxl-conceptslider-age.safetensors'
adapter_name = 'age'

pipe.load_lora_weights(adapter_path, adapter_name=adapter_name)

In [ ]:
# If the strength is not working try to reset your slider to scale=0 (better implementation needed)
pipe.set_adapters(adapter_name, adapter_weights=0)

prompt = "image of a person, realistic, 8k"

negative_prompt = None
slider_scales = [-5, -2.5, 0, 2.5, 5]
seed = 10


images = []



for scale in slider_scales:
    
    sliders_fn = ConceptSliderCallback(slider_strength=.9, 
                                        slider_names=[adapter_name], 
                                        slider_scales=[scale])
    
    image = pipe(
        prompt, 
        negative_prompt= negative_prompt,
        guidance_scale=7,
        num_inference_steps=20, 
        generator=torch.manual_seed(seed),
        callback_on_step_end=sliders_fn,
    ).images[0]
    images.append(image)

display_slider_images(images, slider_scales)

